<a href="https://colab.research.google.com/github/but14/ASL_Detect/blob/main/tieuluanmayhoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-Nearest Neighbor(KNN) Algorithm algorithm


CIFAR-10 là dataset được đánh nhãn do đó đây là máy học có giám sát (Supervised Learning)

In [ ]:
# Import necessary libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
import numpy as np

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Flatten the images for KNN
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Normalize the data
X_train_flattened = X_train_flattened / 255.0
X_test_flattened = X_test_flattened / 255.0

# Initialize KNN model
knn = KNeighborsClassifier(n_neighbors=5)

# Train KNN on the training data
knn.fit(X_train_flattened, y_train.ravel())

# Predict on the test set
y_pred = knn.predict(X_test_flattened)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Test accuracy: 33.98%


In [ ]:

# Load CIFAR-10 dataset and reduce size
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, y_train = X_train[:2000].reshape(2000, -1) / 255.0, y_train[:2000]
X_test, y_test = X_test[:500].reshape(500, -1) / 255.0, y_test[:500]

# Train and evaluate KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train.ravel())
print("Test accuracy:", accuracy_score(y_test, knn.predict(X_test)) * 100, "%")


Test accuracy: 24.0 %


ở đây độ chính xác giảm do n = 3 và KNN test chỉ xữ lí khung hình 500 nên tốc độ nhanh nhưng độ chính xác không cao

#  Principal Component Analysis (PCA) - Dương Nguyễn Anh Quân


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import GridSearchCV

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Flatten the images
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Standardize the data
scaler = StandardScaler()
x_train_flat = scaler.fit_transform(x_train_flat)
x_test_flat = scaler.transform(x_test_flat)

# Use much more PCA components - getting closer to original dimensions
pca = PCA(n_components=2000)  # Significantly increased
x_train_pca = pca.fit_transform(x_train_flat)
x_test_pca = pca.transform(x_test_flat)

# More extensive parameter grid
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5],  # Added more neighbor options
    'weights': ['uniform', 'distance'],
    'metric': ['cosine', 'manhattan'],  # Manhattan distance can work well with high dimensions
    'p': [1, 2],  # L1 and L2 distance metrics
    'leaf_size': [10, 30, 50]  # Can affect performance for large datasets
}

# Run grid search with cross-validation
grid_search = GridSearchCV(
    KNeighborsClassifier(n_jobs=-1),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy'
)

print("Starting grid search...")
grid_search.fit(x_train_pca, y_train.ravel())

# Use best model for final prediction
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(x_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f'\nFinal Test Accuracy: {accuracy * 100:.2f}%')

Starting grid search...
Fitting 3 folds for each of 120 candidates, totalling 360 fits


# Singular Value Decomposition (SVD) - Lê Trần Quốc Bảo

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import cifar10

# 1. Tải dữ liệu CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Chuyển đổi dữ liệu thành dạng vector (kết hợp tất cả kênh RGB)
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# 2. Giảm chiều dữ liệu với SVD
n_components = 100  # Số chiều sau khi giảm, có thể thay đổi để tối ưu
svd = TruncatedSVD(n_components=n_components)
x_train_svd = svd.fit_transform(x_train_flat)
x_test_svd = svd.transform(x_test_flat)

# 3. Huấn luyện và kiểm thử với KNN
k = 5  # Số lượng lân cận (neighbors)
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(x_train_svd, y_train.ravel())

# Dự đoán và đánh giá độ chính xác
y_pred = knn.predict(x_test_svd)
accuracy = accuracy_score(y_test, y_pred)

print(f"Độ chính xác của mô hình KNN với SVD trên CIFAR-10: {accuracy * 100:.2f}%")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step
Độ chính xác của mô hình KNN với SVD trên CIFAR-10: 38.02%


#  Feature Engineering and Selection - Lê Nguyên Thành

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize data to [0, 1]

# Feature Engineering - Compute mean and standard deviation of RGB channels
def rgb_channel_features(images):
    # Calculate mean and standard deviation for each RGB channel
    r_mean = np.mean(images[:, :, :, 0], axis=(1, 2))
    g_mean = np.mean(images[:, :, :, 1], axis=(1, 2))
    b_mean = np.mean(images[:, :, :, 2], axis=(1, 2))
    r_std = np.std(images[:, :, :, 0], axis=(1, 2))
    g_std = np.std(images[:, :, :, 1], axis=(1, 2))
    b_std = np.std(images[:, :, :, 2], axis=(1, 2))
    return np.column_stack((r_mean, g_mean, b_mean, r_std, g_std, b_std))

# Extract channel statistics as features
x_train_features = rgb_channel_features(x_train)
x_test_features = rgb_channel_features(x_test)

# Reshape images to 2D and concatenate with extracted features
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
x_train = np.hstack((x_train, x_train_features))
x_test = np.hstack((x_test, x_test_features))

# Step 3: Standardize data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Step 4: Reduce dimensionality using PCA
pca = PCA(n_components=100)  # Keep 100 principal components
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

# Step 5: Tune KNN parameters and find the best k
best_accuracy = 0
best_k = 0
for k in range(1, 21):  # Test for k from 1 to 20
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train_pca, y_train.ravel())
    y_pred = knn.predict(x_test_pca)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with k={k}: {accuracy * 100:.2f}%")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k

print(f"Best accuracy with KNN: {best_accuracy * 100:.2f}% using k={best_k}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Accuracy with k=1: 38.59%
Accuracy with k=2: 34.61%
Accuracy with k=3: 37.05%
Accuracy with k=4: 37.42%
Accuracy with k=5: 38.44%
Accuracy with k=6: 38.02%
Accuracy with k=7: 38.29%
Accuracy with k=8: 38.33%
Accuracy with k=9: 38.44%
Accuracy with k=10: 38.80%
Accuracy with k=11: 39.00%
Accuracy with k=12: 38.93%
Accuracy with k=13: 39.18%
Accuracy with k=14: 38.69%
Accuracy with k=15: 38.77%
Accuracy with k=16: 38.75%
Accuracy with k=17: 38.65%
Accuracy with k=18: 38.46%
Accuracy with k=19: 38.46%
Accuracy with k=20: 38.56%
Best accuracy with KNN: 39.18% using k=13


#  CLIP với K-NN - Phạm Nguyễn Hoàng Long

In [ ]:
import numpy as np
import torch
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
from tensorflow.keras.datasets import cifar10
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Tải dữ liệu CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


def get_clip_features(images):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Kiểm tra GPU
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16").to(device)  # Chuyển mô hình sang GPU
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")


    processor.feature_extractor.do_rescale = False

    features = []
    for img in images:
        inputs = processor(images=transform(img), return_tensors="pt").to(device)
        with torch.no_grad():
            output = model.get_image_features(**inputs)
        features.append(output.cpu().numpy().flatten())

    return np.array(features)


train_features = get_clip_features(x_train)
test_features = get_clip_features(x_test)


knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(train_features, y_train.flatten())


y_pred = knn.predict(test_features)

# Tính toán độ chính xác
accuracy = accuracy_score(y_test.flatten(), y_pred)
print(f"Overall accuracy on CIFAR-10: {accuracy * 100:.2f}%")


Overall accuracy on CIFAR-10: 93.91%


# Data Augmentation - Trịnh Ngọc Nhất

In [ ]:
# Import necessary libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Flatten the images for KNN (before augmentation)
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Normalize the data
X_train_flattened = X_train_flattened / 255.0
X_test_flattened = X_test_flattened / 255.0

# Set up data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,      # Randomly rotate images by 15 degrees
    width_shift_range=0.1,  # Shift images horizontally by 10%
    height_shift_range=0.1, # Shift images vertically by 10%
    horizontal_flip=True    # Randomly flip images horizontally
)

# Fit the generator on the training data
datagen.fit(X_train)

# Augment data by generating a subset of augmented images for KNN
X_augmented, y_augmented = [], []
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=5000, shuffle=False):
    X_augmented.append(X_batch)
    y_augmented.append(y_batch)
    if len(X_augmented) * 5000 >= 100000:  # Limit the augmented set to 10,000 samples
        break

# Convert the augmented data to numpy arrays
X_augmented = np.concatenate(X_augmented).reshape(-1, 32 * 32 * 3) / 255.0
y_augmented = np.concatenate(y_augmented).ravel()

# Combine the original and augmented data
X_train_combined = np.concatenate((X_train_flattened, X_augmented), axis=0)
y_train_combined = np.concatenate((y_train.ravel(), y_augmented), axis=0)

# Initialize KNN model
knn = KNeighborsClassifier(n_neighbors=5)

# Train KNN on the combined training data
knn.fit(X_train_combined, y_train_combined)

# Predict on the test set
y_pred = knn.predict(X_test_flattened)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy with data augmentation: {accuracy * 100:.2f}%")


Test accuracy with data augmentation: 35.57%
